# Fake currency detection using ResNet50 model

In [1]:
#import all necessary modules
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array 
from tensorflow.keras.layers import Dense,Activation,Flatten,Dropout
from tensorflow.keras.models import Sequential,Model,load_model
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#define height and width of the image
height=300
width=300

#create a ResNet50 model instance without the top layer as we will add our own top layer
base_model=ResNet50(weights='imagenet',include_top=False,input_shape=(height,width,3))

In [3]:
#define directory containing training and validation data
train_dir="dataset/training"
validation_dir="dataset/validation"

#number of batches the data has to be divided into
batch_size=8

#create datagen and generator to load the data from training directory
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input,rotation_range=90,horizontal_flip=True,vertical_flip=True)
train_generator=train_datagen.flow_from_directory(train_dir,target_size=(height,width),batch_size=batch_size)

#create datagen and generator to load the data from validation directory
validation_datagen=ImageDataGenerator(preprocessing_function=preprocess_input,rotation_range=90,horizontal_flip=True,vertical_flip=True)
validation_generator=validation_datagen.flow_from_directory(validation_dir,target_size=(height,width),batch_size=batch_size)

Found 2774 images belonging to 2 classes.
Found 592 images belonging to 2 classes.


In [4]:
def build_finetune_model(base_model,dropout,fc_layers,num_classes):
    for layer in base_model.layers:
        layer.trainable=False

    x=base_model.output
    x=Flatten()(x)
    for fc in fc_layers:
        x=Dense(fc,activation='relu')(x)
        x=Dropout(dropout)(x)
    
    predictions=Dense(num_classes,activation='softmax')(x)

    finetune_model=Model(inputs=base_model.input,outputs=predictions) 
    
    return finetune_model

class_list=['Real','Fake'] #the labels of our data
FC_Layers=[1024,1024]
dropout=0.5

finetune_model=build_finetune_model(base_model,dropout=dropout,fc_layers=FC_Layers,num_classes=len(class_list))

In [5]:
num_epochs=50
num_train_images=2774

In [6]:

#checkpoint in case anything goes wrong
checkpoint=ModelCheckpoint("Final_model.h5",monitor='val_accuracy',verbose=1,save_best_only=True,save_weights_only=False,mode='auto',save_freq='epoch')
early=EarlyStopping(monitor='val_accuracy',min_delta=0,patience=40,verbose=1,mode="auto")

#compile the model before using
optimizer = optimizers.SGD(learning_rate=0.000001, momentum=0.9)
finetune_model.compile(loss="categorical_crossentropy",optimizer=optimizer, metrics=['accuracy'])

#train the model
finetune_model.fit(train_generator,steps_per_epoch=num_train_images//batch_size,epochs=num_epochs,validation_data=validation_generator,validation_steps=1,callbacks=[checkpoint,early])

#save the model
finetune_model.save_weights("Final_model.h5")

c:\Users\PRATYAOY SARKAR\FinalYrProject\myenv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
346/346 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.5354 - loss: 1.5772

c:\Users\PRATYAOY SARKAR\FinalYrProject\myenv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()



Epoch 1: val_accuracy improved from None to 0.62500, saving model to Final_model.h5


346/346 ━━━━━━━━━━━━━━━━━━━━ 1559s 4s/step - accuracy: 0.5528 - loss: 1.4475 - val_accuracy: 0.6250 - val_loss: 0.7437
Epoch 2/50
  1/346 ━━━━━━━━━━━━━━━━━━━━ 1:38:39 17s/step - accuracy: 0.5000 - loss: 1.3193

...


Epoch 2: val_accuracy did not improve from 0.62500
346/346 ━━━━━━━━━━━━━━━━━━━━ 21s 11ms/step - accuracy: 0.5000 - loss: 1.3193 - val_accuracy: 0.6250 - val_loss: 1.0146
Epoch 3/50
346/346 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.6037 - loss: 1.1993
Epoch 3: val_accuracy did not improve from 0.62500
346/346 ━━━━━━━━━━━━━━━━━━━━ 1114s 3s/step - accuracy: 0.6106 - loss: 1.1529 - val_accuracy: 0.6250 - val_loss: 0.7440
Epoch 4/50
  1/346 ━━━━━━━━━━━━━━━━━━━━ 13:39 2s/step - accuracy: 0.6250 - loss: 1.4175
Epoch 4: val_accuracy improved from 0.62500 to 0.75000, saving model to Final_model.h5


346/346 ━━━━━━━━━━━━━━━━━━━━ 16s 38ms/step - accuracy: 0.6250 - loss: 1.4175 - val_accuracy: 0.7500 - val_loss: 0.5554
Epoch 5/50
346/346 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6636 - loss: 0.9129
Epoch 5: val_accuracy did not improve from 0.75000
346/346 ━━━━━━━━━━━━━━━━━━━━ 732s 2s/step - accuracy: 0.6638 - loss: 0.8975 - val_accuracy: 0.5000 - val_loss: 0.7948
Epoch 6/50
  1/346 ━━━━━━━━━━━━━━━━━━━━ 12:06 2s/step - accuracy: 0.6250 - loss: 1.0137
Epoch 6: val_accuracy did not improve from 0.75000
346/346 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.6250 - loss: 1.0137 - val_accuracy: 0.2500 - val_loss: 1.1868
Epoch 7/50
346/346 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6618 - loss: 0.9331
Epoch 7: val_accuracy did not improve from 0.75000
346/346 ━━━━━━━━━━━━━━━━━━━━ 748s 2s/step - accuracy: 0.6725 - loss: 0.8942 - val_accuracy: 0.7500 - val_loss: 0.5553
Epoch 8/50
  1/346 ━━━━━━━━━━━━━━━━━━━━ 12:40 2s/step - accuracy: 0.7500 - loss: 0.4696
Epoch 8: val_accuracy improve

346/346 ━━━━━━━━━━━━━━━━━━━━ 14s 35ms/step - accuracy: 0.7500 - loss: 0.4696 - val_accuracy: 0.8750 - val_loss: 0.4622
Epoch 9/50
346/346 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7048 - loss: 0.7591
Epoch 9: val_accuracy improved from 0.87500 to 1.00000, saving model to Final_model.h5


346/346 ━━━━━━━━━━━━━━━━━━━━ 743s 2s/step - accuracy: 0.7097 - loss: 0.7495 - val_accuracy: 1.0000 - val_loss: 0.1752
Epoch 10/50
  1/346 ━━━━━━━━━━━━━━━━━━━━ 36:47 6s/step - accuracy: 0.7500 - loss: 0.9181
Epoch 10: val_accuracy did not improve from 1.00000
346/346 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - accuracy: 0.7500 - loss: 0.9181 - val_accuracy: 0.7500 - val_loss: 0.4556
Epoch 11/50
346/346 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7202 - loss: 0.7247
Epoch 11: val_accuracy did not improve from 1.00000
346/346 ━━━━━━━━━━━━━━━━━━━━ 744s 2s/step - accuracy: 0.7238 - loss: 0.7186 - val_accuracy: 0.6250 - val_loss: 0.8708
Epoch 12/50
  1/346 ━━━━━━━━━━━━━━━━━━━━ 21:12 4s/step - accuracy: 0.6250 - loss: 0.5172
Epoch 12: val_accuracy did not improve from 1.00000
346/346 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.6250 - loss: 0.5172 - val_accuracy: 0.6250 - val_loss: 0.7156
Epoch 13/50
346/346 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7465 - loss: 0.6508
Epoch 13: val_accuracy d

KeyboardInterrupt: 